In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.integrate as integrate
import numpy.linalg as linalg
from multipledispatch import dispatch

In [2]:
N=1000
startR=-0.25/au2nm
endR=0.25/au2nm
N=1000
sites=N+1
a=0.0529/au2nm
deltaX=np.abs(startR-endR)/(N) #grid size


eps=0.5968404*au2nm*au2ev
n=1
sigma=1
valueAtZero=1/(2*np.pi*a*eps)

In [ ]:
class Point:
    def __init__(self, x, y):
        self.x=x
        self.y=y
        
class Triangle:
    def __init__(self, p_1, p_2, p_3):
        self.p_1=p_1
        self.p_2=p_2
        self.p_3=p_3
    def area():
        return (0.5)*(self.p2.y+p1.y)*(self.p1.x-p2.x)+0.5*(self.p1.y+self.p3.y)*(self.p3.x-self.p1.x)-0.5*(self.p2.y+self.p3.y)*(self.p3.x-self.p2.x)  
    
    def is_in(x, y):
        Area=self.area();
        s = 1/(2*Area)*(self.p_1.y*self.p_1.x - self.p_1.x*self.p_1.y + (p2y - p0y)*px + (p0x - p2x)*py);
        t = 1/(2*Area)*(p0x*p1y - p0y*p1x + (p0y - p1y)*px + (p1x - p0x)*py);
        return 0<s and s<1 and t>0 and t<1 and t+s<1
    
    def evaluate(x, y):
        return 1-x-y if
class FiniteElement:
    def _init_(tri_1, tri_2, tri_3):
        self.tri_1=tri_1
        self.tri_2=tri_2
        self.tri_3=tri_3
    
    def evaluate(x, y):
        

In [ ]:
def tri_area(p1,p2,p3):


def linear_element(x, p_1, p_2):
    return ((p_1.y-p_2.y)/(p_1.x-p_2.x))(x-p_1.x)+p_1.y

def triangle(x, y):
    